# Infinite-Dimensional Bayesian Filtering for Detection of Quasiperiodic Phenomena in Spatiotemporal data

$\frac{\partial^2}{\partial t^2}x(\mathbf{u},t) + \mathcal{A}\frac{\partial}{\partial t}x(\mathbf{u},t) + \mathcal{B}x(\mathbf{u},t) = \xi(\mathbf{u},t)$

$C(\mathbf{u},\mathbf{u}') = \kappa_\xi(\mathbf{u},\mathbf{u}')\delta(t-t')$

$\kappa_\xi(\mathbf{u},\mathbf{u}') = \sigma^2\frac{2^{1-\nu}}{\Gamma\nu}\left(\sqrt{2\nu}\frac{\|\mathbf{u}-\mathbf{u}'\|}{\mathcal{l}}\right)^\nu \mathcal{K}_\nu\left(\sqrt{2\nu}\frac{\|\mathbf{u}-\mathbf{u}'\|}{\mathcal{l}}\right)$

In [1]:
# Resonator parameters
𝛾, χ = 1., 0.01

# Covariance parameters
ν, l, σₖ = 3./2., 0.1, 0.25

ω = 6.#Hz
;# suppress output

## Define Differential Operators

$\nabla^2f(\mathbf{x},t)$ is the Laplacian operator, $\nabla\cdot\nabla f(\mathbf{x},t) = \sum_i\frac{\partial^2}{\partial x_i^2}f(\mathbf{x},t)$

$\nabla^4f(\mathbf{x},t)$ is the biharmonic operator, $\nabla^2(\nabla^2f(\mathbf{x},t)) = \sum_i\frac{\partial^4}{\partial x_i^4}f(\mathbf{x},t) + \sum_i\sum_j\frac{\partial^4}{\partial x_i^2\partial x_j^2}f(\mathbf{x},t)$

In [2]:
using ForwardDiff # for hessian

∇² = f -> (x -> trace(ForwardDiff.hessian(f,x))) # laplacian
∇⁴ = f -> (x -> ∇²(∇²(f))(x))         # biharmonic

(::#7) (generic function with 1 method)

### Check differential operators

Confirm autodifferention against test function $f(\mathbf{x}) = \sin x_1 + x_1\cos x_2$

Where $\nabla^2f = -\sin x_1 - x_1\cos x_2$ and $\nabla^4f = f(\mathbf{x}) = \sin x_1 + x_1\cos x_2$

In [3]:
f   = x -> sin(x[1]) + x[1]*cos(x[2])

∇²f = x -> -sin(x[1]) - x[1]*cos(x[2])
∇⁴f = x -> sin(x[1])  + x[1]*cos(x[2])

testpoints = [x = (100 * rand(2)) - 50 for _ in 1:100]

using Base.Test

@testset "Tests" begin
    @testset "Laplacian" for x ∈ testpoints
        @test ∇²(f)(x) ≈ ∇²f(x) atol=eps()
    end
    @testset "Biharmonic" for x ∈ testpoints
        @test ∇⁴(f)(x) ≈ ∇⁴f(x) atol=eps()
    end
end;

Test Summary: | Pass  Total
Tests         |  200    200


## Model Setup

Operator definitions from _Solin and Särkkä_:

In [17]:
𝒜 = f -> (x -> 𝛾 - χ*∇²(f)(x))
 
ℬ = f -> (x -> 𝛾^2/2 - 𝛾*χ*∇²(f)(x) + χ^2/2 * ∇⁴(f)(x) + ω^2)

#ℱ = f -> (x -> [ 0. f[2](x) ; -ℬ(f[1])(x) -𝒜(f[2])(x)])
#currying
ℱ = f₁ -> (f₂ -> (x -> [ 0. f₂(x) ; -ℬ(f₁)(x) -𝒜(f₂)(x)]))

𝐋 = [0 1]'

#𝒰 = f -> (τ -> (x -> expm(τ * ℱ(f)(x))*[1.;1.]))
# currying
𝒰₁ = f₁ -> (f₂ -> (τ -> (x -> expm(τ * ℱ(f₁)(f₂)(x))[1])))
𝒰₂ = f₁ -> (f₂ -> (τ -> (x -> expm(τ * ℱ(f₁)(f₂)(x))[2])))

(::#235) (generic function with 1 method)

# Create sample data

In [22]:
include("GP.jl")

nₜ = 10
𝐭 = linspace(0., 1., nₜ)
 
𝐶 = u -> sample_kernel(u, 𝑘 = kern_matern, θ = Dict("nu"=>ν, "lengthscale" => l, "variance" => σₖ^2))

x₀ = u -> cos.(u) + 𝐶(u)

xₜ = [u -> x₀(u); u -> -𝒜(x₀)(u)]

𝒙 = Array{Function}(2,nₜ)
𝒙[:,1] = xₜ

;

In [23]:
for k ∈ 2:nₜ
    Δₜ = 𝐭[k] - 𝐭[k-1]
    
    x₋ = xₜ
    
    xₜ = [ u -> 𝒰₁(x₋[1])(x₋[2])(Δₜ)(u) + 𝐶(u) ;
           u -> 𝒰₂(x₋[1])(x₋[2])(Δₜ)(u) + 𝐶(u) ]
    
    𝒙[:,k] = xₜ
end

In [24]:
using PyPlot

L, nₓ = 2π, 100       # Domain limits and mesh resolution
Ω = linspace(-L, L, nₓ) # Spatial domain

𝐟 = zeros(nₓ, nₜ)

for k ∈ 1:nₜ
    println(k)
    𝐟[:,k] = 𝒙[:,k][1](Ω)
end

1
2


LoadError: [91mMethodError: no method matching svdfact!(::Array{ForwardDiff.Dual{ForwardDiff.Tag{##371#372,Float64},ForwardDiff.Dual{ForwardDiff.Tag{##371#372,Float64},Float64,10},10},2}; thin=true)[0m
Closest candidates are:
  svdfact!([91m::Base.LinAlg.AbstractTriangular[39m) at linalg/triangular.jl:2204[91m got unsupported keyword argument "thin"[39m
  svdfact!([91m::Union{Base.ReshapedArray{T<:Union{Complex{Float32}, Complex{Float64}, Float32, Float64},2,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:Union{DenseArray, SubArray{T,N,P,I,true} where I<:Tuple{Union{Base.Slice, UnitRange},Vararg{Any,N} where N} where P where N where T}, DenseArray{T<:Union{Complex{Float32}, Complex{Float64}, Float32, Float64},2}, SubArray{T<:Union{Complex{Float32}, Complex{Float64}, Float32, Float64},2,A,I,L} where L} where I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex, Int64, Range{Int64}},N} where N} where A<:Union{Base.Resha

In [12]:
𝒙[:,2][1]

(::#112) (generic function with 1 method)